In [139]:
nlp_model = 'ldabool'

from experiments._60_nlpmodels.fit_social_nlp_models import *

from sklearn.metrics import f1_score

f1s_social = load_nlp_selected_users()

nlp_uids = [x[0] for x in f1s_social]

uid_pos = {uid: pos for pos, uid in enumerate(nlp_uids)}

In [140]:
# def scale(X_train, X_test):
#     train_size = X_train.shape[0]
#     X = np.concatenate((X_train, X_test))
#     scaler = StandardScaler()
#     scaler.fit(X)
#     X = scaler.transform(X)
#     X_train = X[:train_size,:]
#     X_test = X[train_size:,:]

#     return X_train, X_test, scaler

# f1s_combined = {}

# for n_topics in [10, 20]:
#     f1s_n = {'train': [], 'valid': [], 'test': []}
#     nlp_feats = pd.read_pickle('alltweets_es_%s%d.pickle' % (nlp_model, n_topics))

#     for uid, f1 in f1s_social:
#         uid = int(uid)
#         clf = load_model(uid, 'svc' + nlp_model, n_topics=n_topics, subfolder='social_nlp')

#         # Train classifiers only if not created already
#         print "==============================" 
#         print "Processing %d ( f1 %.2f %%)" % (uid, 100 * f1)

#         X_train, X_valid, X_test, y_train, y_valid, y_test = load_dataframe(uid)

#         X_train_nlp = nlp_feats.loc[X_train.index]
#         X_valid_nlp = nlp_feats.loc[X_valid.index]
#         X_test_nlp = nlp_feats.loc[X_test.index]

#         X_train_combined = np.hstack((X_train, X_train_nlp))
#         X_valid_combined = np.hstack((X_valid, X_valid_nlp))
#         X_test_combined = np.hstack((X_test, X_test_nlp))

#         X_train_combined, X_valid_combined, scaler = scale(X_train_combined, X_valid_combined)
#         # Aplicar mismo scaling a X_test
#         X_test_combined = scaler.transform(X_test_combined)
        
#         y_true, y_pred = y_train, clf.predict(X_train_combined)
#         f1s_n['train'].append(f1_score(y_true, y_pred))
        
#         y_true, y_pred = y_valid, clf.predict(X_valid_combined)
#         f1s_n['valid'].append(f1_score(y_true, y_pred))
        
#         y_true, y_pred = y_test, clf.predict(X_test_combined)
#         f1s_n['test'].append(f1_score(y_true, y_pred))
        
#     f1s_combined[n_topics] = f1s_n

In [141]:
# with open('f1s_combined_%s.json' % nlp_model, 'w') as f:
#     json.dump(f1s_combined, f)

In [142]:
with open('f1s_combined_%s.json' % nlp_model) as f:
    f1s_combined = json.load(f)

# Comparación sobre set de prueba

In [143]:
from bokeh.charts import Line, show, output_notebook

In [144]:
output_notebook()

Loading BokehJS ...

In [145]:
f1s_test_social = json.load(open('../_1_one_user_learn_neighbours/f1s_testv_svc.json'))
f1s_test_social = {u: f for u, f in f1s_test_social.items() if u in nlp_uids}

In [146]:
sorted_uids = [u for u, f in sorted(f1s_test_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

In [147]:
f1s_10 = np.array([f1s_combined['10']['test'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_20 = np.array([f1s_combined['20']['test'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_s = np.array([f1s_test_social[sorted_uids[i]] for i in indices])

In [148]:
# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+ldabool10": f1s_10,
    "social+ldabool20": f1s_20
}

labels = 'social social+ldabool10 social+ldabool20'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuario', ylabel='puntaje f1', legend=True)

# output_file("line_single.html", title="line_single.py example")

show(line)

In [149]:
sum(f1s_10 > f1s_s)

8

In [150]:
sum(f1s_20 > f1s_s)

7

In [151]:
(np.mean(f1s_10), np.mean(f1s_20))

(0.73218909052513992, 0.71275150813029986)

In [152]:
for f in [f1s_10, f1s_20]:
    better = f > f1s_s
    mean_improvement = np.mean(f[better] - f1s_s[better])
    print mean_improvement

0.0239536041184
0.0250724620218


# Comparación en set de entrenamiento

In [153]:
f1s_train_social = json.load(open('../_1_one_user_learn_neighbours/f1s_train_svc.json'))

In [154]:
f1s_train_social = {u: f for u, f in f1s_train_social.items() if u in nlp_uids}

In [155]:
sorted_uids = [u for u, f in sorted(f1s_train_social.items(), key=lambda x:x[1])]
indices = range(len(sorted_uids))

In [156]:
f1s_10 = np.array([f1s_combined['10']['train'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_20 = np.array([f1s_combined['20']['train'][uid_pos[sorted_uids[i]]] for i in indices])
f1s_s = np.array([f1s_train_social[sorted_uids[i]] for i in indices])

In [157]:
# build a dataset where multiple columns measure the same thing
data = {
    "social": f1s_s,
    "social+ldabool10": f1s_10,
    "social+ldabool20": f1s_20
}

labels = 'social social+ldabool10 social+ldabool20'.split()

# create a line chart where each column of measures receives a unique color and dash style
line = Line(data, y=labels,
            dash=labels,
            color=labels,
            legend_sort_field = 'color',
            legend_sort_direction = 'descending',
            xlabel='usuario', ylabel='puntaje f1', legend=True)

# output_file("line_single.html", title="line_single.py example")

show(line)

In [158]:
sum(f1s_10 > f1s_s)

32

In [159]:
sum(f1s_20 > f1s_s)

32

In [160]:
(np.mean(f1s_10), np.mean(f1s_20))

(0.8917796346688398, 0.91879821536731221)

In [161]:
for f in [f1s_10, f1s_20]:
    better = f > f1s_s
    mean_improvement = np.mean(f[better] - f1s_s[better])
    print mean_improvement

0.174321835558
0.201192540236
